In [7]:
import pandas as pd, numpy as np

In [65]:
def residual_valuation(current_book_value, earnings_forecast, dividends_forecast, required_return, long_term_growth):
    """Calculate the value of a stock using residual earnings valuation

    Args:
        current_book_value (float): Current book value of equity
        earnings_forecast (list): Future projected earnings
        dividends_forecast (list): Future projected dividends
        required_return (float): required return on the stock in decimals
        long_term_growth (float): Long term terminal growth rate in residual earnings

    Returns:
        dict: contains 'total_value' of firm, with its valuation components
    """
    
    df = pd.DataFrame({'earnings': [np.nan] + earnings_forecast,
                      'dividends': [np.nan] + dividends_forecast,
                      'book_value': [current_book_value] + [np.nan] * len(earnings_forecast)
                      })
    periods = len(earnings_forecast)
    
    for t in range(1, periods+1):
        df['book_value'][t] = df['book_value'][t-1] + df['earnings'][t] - df['dividends'][t]
        
    df['charge'] = df['book_value'].shift(1) * required_return
    df['discount_factor'] = (1+required_return)**(-1 * df.index.values)
    df['residual_earnings'] = df['earnings'] - df['charge']
    df['pv_re'] = df['residual_earnings'] * df['discount_factor']
    
    continuing_value = df['residual_earnings'].iloc[-1] * (1 + long_term_growth) / (required_return - long_term_growth)
    pv_cv = continuing_value * df['discount_factor'].iloc[-1]
    
    return {'total_value': current_book_value + df['pv_re'].sum() + pv_cv,
            'book_value': current_book_value,
            'non_terminal_re': df['pv_re'].sum(),
            'pv_cv': pv_cv}

residual_valuation(current_book_value = 20.15, 
                   earnings_forecast = [4.29, 4.78, 5.31, 5.89, 6.54],
                   dividends_forecast = [1.16, 1.29, 1.43, 1.59, 1.77],
                   required_return = 0.09,
                   long_term_growth = 0.04)

{'total_value': 77.23505454282589,
 'book_value': 20.15,
 'non_terminal_re': 11.196259054399437,
 'pv_cv': 45.88879548842646}

In [66]:
residual_valuation(current_book_value = 9.96, 
                   earnings_forecast = [2.39, 3.45, 2.28, 2.00, 1.71],
                   dividends_forecast = [1.06, 1.12, 1.16, 1.22, 1.24],
                   required_return = 0.11,
                   long_term_growth = 0.0)

{'total_value': 13.756078287208394,
 'book_value': 9.96,
 'non_terminal_re': 3.7809722534032666,
 'pv_cv': 0.01510603380512648}

# Models
FCFF
FCFE
DDM
RNOI
RNI

In [ ]:
def ddm(r_e, g_t, div):
    # Discounts array of dividends based on required return and terminal growth
    dc_factor = np.power(1+r_e, -np.arange(1, len(div)+1))
    terminal_value = div[-1] * (1+g_t) / (r_e - g_t) * dc_factor[-1]
    return (np.array(div) * dc_factor).sum() + terminal_value

ddm(0.12, 0.05, [0.03, 0.04, 0.06, 0.07, 0.09, 0.11, 0.13, 0.17, 0.2, 0.22, 0.28, 0.32, 0.4, 0.48, 0.56, 0.6, 0.72])